# Prueba - Fundamentos Data Science
---
Martes 19 de mayo de 2020, Santiago de Chile

### Sakura SPA
**Miembros de la Célula:** 

Susana Arce

Fabiola Aravena

Rodrigo Pereira

--- 

**Administrador de Contrato:** Gonzalo Seguel

**Sponsor:** Andrea Villaroel

---  

#### Objetivo:
Nuestro sponsor posee un cátalogo de contenido audiovisual de 12.294 títulos de Anime, requiere poder efecturar recomendaciones de qué ver a los usuarios en base a otros Anime han sido de su gusto.

#### Propuesta:
Aplicación web en donde el usuario seleccione de una lista de Anime propuestos los que han sido de su gusto, y en base a ello se le entregue una lista de otras alternativas que sean a fin con sus preferencias.

---
## Búsqueda de Posters asociados a los anime

Este registro, muestra la labor realizar para hacer una ingesta de la data asociada a url en donde se encuentran las portadas de cada título de anime, esta data adicional se ingesto para ser utilizada en la fase de producción de la plataforma.


In [1]:
# Importación de librerías para procesamiento de datos.
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

import factor_analyzer as factor
import missingno as msngo

import warnings
warnings.filterwarnings(action="ignore")

In [32]:
subgenre = ["Action","Adventure","Cars","Comedy","Dementia","Demons","Drama","Ecchi","Fantasy","Game","Harem","Hentai","Historical","Horror","Josei","Kids","Magic","MartialArts","Mecha","Military","Music","Mystery","Parody","Police","Psychological","Romance","Samurai",
"School","Sci-Fi","Seinen","Shoujo","ShoujoAi","Shounen","ShounenAi","SliceofLife","Space","Sports","Supernatural","SuperPower","Thriller","Vampire","Yaoi","Yuri"]

---
### Recogida de datos

Es importante indicar que existe un límite comercial de 5.000 consultas diarias a la API que nos proporciona los posters de anime


In [30]:
#Importamos los data sets de las ingestas anteriores realizadas
df_t1 = pd.read_csv("titulos_con_url_1.csv", sep=";") 
df_t2 = pd.read_csv("titulos_con_url_2.csv", sep=";") 
df_t3 = pd.read_csv("titulos_con_url_3.csv", sep=";") 
df_t44 = pd.read_csv("titulos_con_url_44.csv", sep=";") 
df_t45 = pd.read_csv("titulos_con_url_45.csv", sep=";") 
df_t46 = pd.read_csv("titulos_con_url_46.csv", sep=";") 

In [111]:
df_titulos_generos = pd.read_csv("titulos_generos_VF.csv", sep=";") 
df_titulos_generos.head()

,Unnamed: 0,index,anime_id,name,genre,type,episodes,rating,members,log_members,...,ShounenAi,SliceofLife,Space,Sports,Supernatural,SuperPower,Thriller,Vampire,Yaoi,Yuri
0,0,5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10.0,9.15,93351,11.444122,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110.0,9.11,80679,11.298234,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,11,28851,Koe no Katachi,"Drama, School, Shounen",Movie,1.0,9.05,102733,11.539889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,33,28957,Mushishi Zoku Shou: Suzu no Shizuku,"Adventure, Fantasy, Historical, Mystery, Seine...",Movie,1.0,8.75,32266,10.381769,...,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,4,37,31757,Kizumonogatari II: Nekketsu-hen,"Action, Mystery, Supernatural, Vampire",Movie,1.0,8.73,34347,10.444270,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN


In [118]:
df_titulos_generos.shape

(7492, 54)

---
### Procesamiento de Urls

Se analizan los datos de los registros de días anteriores

In [6]:
df_titulos_generos["url"] = None

In [112]:
df_t46["url"].isnull().value_counts()

True     7805
False    4489
Name: url, dtype: int64

In [34]:
df_t46["new_url"]=None

In [52]:
df_t46["new_url"] = np.where( pd.isnull(df_t46["url"]),df_t1["url"],df_t46["url"])
df_t46["new_url"].isnull().value_counts()

True     7805
False    4489
Name: new_url, dtype: int64

In [53]:
df_t46["new_url"] = np.where( pd.isnull(df_t46["new_url"]),df_t2["url"],df_t46["new_url"])
df_t46["new_url"].isnull().value_counts()

True     7805
False    4489
Name: new_url, dtype: int64

In [54]:
df_t46["new_url"] = np.where( pd.isnull(df_t46["new_url"]),df_t3["url"],df_t46["new_url"])
df_t46["new_url"].isnull().value_counts()

True     7805
False    4489
Name: new_url, dtype: int64

In [55]:
df_t46["new_url"] = np.where( pd.isnull(df_t46["new_url"]),df_t45["url"],df_t46["new_url"])
df_t46["new_url"].isnull().value_counts()

True     7805
False    4489
Name: new_url, dtype: int64

---

In [113]:
df_transfer = df_t46.loc[:,["anime_id","new_url"]]
df_transfer.head()

,anime_id,new_url
0,32281,https://imdb-api.com/images/original/MV5BODRmZ...
1,5114,https://imdb-api.com/images/original/MV5BZmEzN...
2,28977,https://imdb-api.com/images/original/MV5BNzM4Y...
3,9253,https://imdb-api.com/images/original/MV5BYmJhM...
4,9969,https://imdb-api.com/images/original/MV5BNzM4Y...


In [114]:
df_transfer.shape

(12294, 2)

In [115]:
# Se cargan los registros existentes al data set de df_titulos
df_titulos_generos = pd.merge(right=df_titulos_generos,left=df_transfer)
df_titulos_generos.head()

,anime_id,new_url,Unnamed: 0,index,name,genre,type,episodes,rating,members,...,ShounenAi,SliceofLife,Space,Sports,Supernatural,SuperPower,Thriller,Vampire,Yaoi,Yuri
0,32935,https://imdb-api.com/images/original/MV5BNzQ1M...,0,5,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10.0,9.15,93351,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,820,NaN,1,7,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110.0,9.11,80679,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28851,https://imdb-api.com/images/original/MV5BZGRkO...,2,11,Koe no Katachi,"Drama, School, Shounen",Movie,1.0,9.05,102733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28957,https://imdb-api.com/images/original/MV5BMjM5N...,3,33,Mushishi Zoku Shou: Suzu no Shizuku,"Adventure, Fantasy, Historical, Mystery, Seine...",Movie,1.0,8.75,32266,...,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,31757,https://imdb-api.com/images/original/MV5BZjVjN...,4,37,Kizumonogatari II: Nekketsu-hen,"Action, Mystery, Supernatural, Vampire",Movie,1.0,8.73,34347,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN


In [116]:
df_titulos_generos["new_url"].isnull().value_counts()

True     4689
False    2803
Name: new_url, dtype: int64

In [117]:
df_titulos_generos.shape

(7492, 54)

In [157]:
df_titulos = df_titulos_generos[df_titulos_generos["new_url"].isnull()].reset_index()
df_titulos.shape

(4689, 55)

----

**Ingestar imagen de los anime**

In [10]:
#solo ejecutar una vez
#df_titulos["url"] = None

In [11]:
import json
import requests

result = 0 #solo ejecutar una vez --> cantidad de títulos con respuesta no vacía
url_cant = 0 #solo ejecutar una vez --> cantidad de títulos respondidos por la API
contador = 0 #solo ejecutar una vez -- cantidad de títulos consultados


## Ejecutar una celda por día

In [19]:
for name in df_titulos["name"]:
    year = 0
    namee = name.replace("/","%20")
    namee = name.replace(" ","%20")
    url = "https://imdb-api.com/ja/API/SearchSeries/k_h2JDsS1e/"+namee
    if list(df_titulos[df_titulos["name"] == name]["type"])[0] == "Movie":
        url = "https://imdb-api.com/ja/API/SearchMovie/k_h2JDsS1e/"+namee

    response = requests.request("GET", url)
 #   print(response.status_code)
    if response.status_code == 200:
        if json.loads(response.text)["results"] is not None:
            if json.loads(response.text)["results"] != []:
                result = result +1
                print(namee)
                print(url)
                for i in json.loads(response.text)["results"]:
                    df_titulos.loc[df_titulos["name"]==name,"url"] = i["image"]
                    print(i["image"])
                    if  i["description"][1:5].isdigit():
                        yea = int(i["description"][1:5])
                        if yea > year:
                            year = yea
                            df_titulos.loc[df_titulos["name"]==name,"url"] = i["image"]
#        print(df_titulos.loc[df_titulos["name"]==name,"url"])
#        print("-------------------------------------------------------")
        url_cant = url_cant +1
    contador = contador +1

Bonobono%20(TV)
https://imdb-api.com/ja/API/SearchSeries/k_h2JDsS1e/Bonobono%20(TV)
https://imdb-api.com/images/original/MV5BMDQzOTVhMDQtMTNkMi00MzE1LTg1ZjktYTYyY2U5OWZjZWMwXkEyXkFqcGdeQXVyNTY0NDkzNDc@._V1_Ratio0.7273_AL_.jpg
FlashBack
https://imdb-api.com/ja/API/SearchSeries/k_h2JDsS1e/FlashBack
https://imdb-api.com/images/original/MV5BYjc4MzE2NzctNzQyMS00ZmRhLThhOTEtM2NkYjkzZTFjOWZiXkEyXkFqcGdeQXVyNjgzMjQ0MTA@._V1_Ratio0.7273_AL_.jpg
Interlude
https://imdb-api.com/ja/API/SearchSeries/k_h2JDsS1e/Interlude
https://imdb-api.com/images/original/MV5BNjkyZGRjMmItOTY3YS00ZDc5LWFiMGItMGI4Njk5ZTk3NTRlXkEyXkFqcGdeQXVyMTA1OTEwNjE@._V1_Ratio0.7273_AL_.jpg
Hairy%20Tale
https://imdb-api.com/ja/API/SearchSeries/k_h2JDsS1e/Hairy%20Tale
https://imdb-api.com/images/original/MV5BMmE4NmE1MjctN2E5MC00MWMyLWE0MTYtNjMyYjIwMmNhMGE3XkEyXkFqcGdeQXVyMTAwMzM3NDI3._V1_Ratio0.7273_AL_.jpg
Piano
https://imdb-api.com/ja/API/SearchSeries/k_h2JDsS1e/Piano
https://imdb-api.com/images/original/MV5BNTBmZDFmNDctMWI3MS00M

In [20]:
print("Cant. de consultas ejecutadas:")
print(contador )
print("Cant. de títulos con conexión establecida por la API:")
print(url_cant)
print("Cant. de títulos con resultados no vacíos")
print(result)
print("Cant. último título consultado:")
print(name)
print("Ubicación último título consultado")
print(np.where(df_titulos["name"]==name))

Cant. de consultas ejecutadas:
4917
Cant. de títulos con conexión establecida por la API:
4850
Cant. de títulos con resultados no vacíos
11
Cant. último título consultado:
Sakura no Mori
Ubicación último título consultado
(array([4557]),)


In [21]:
df_titulos["url"].value_counts()

https://imdb-api.com/images/original/nopicture.jpg                                                                                               4
https://imdb-api.com/images/original/MV5BNTBmZDFmNDctMWI3MS00MDE2LThiN2EtZDBjYzQxNGFiNWExXkEyXkFqcGdeQXVyNjU3MzA0NjE@._V1_Ratio0.7273_AL_.jpg    1
https://imdb-api.com/images/original/MV5BMmE4NmE1MjctN2E5MC00MWMyLWE0MTYtNjMyYjIwMmNhMGE3XkEyXkFqcGdeQXVyMTAwMzM3NDI3._V1_Ratio0.7273_AL_.jpg    1
https://imdb-api.com/images/original/MV5BMDJhNDQ1MjktM2UzZi00M2M4LWIzNDAtZDQwY2Q1NDllNmM0XkEyXkFqcGdeQXVyMjY0MTQ0NjY@._V1_Ratio0.7273_AL_.jpg    1
https://imdb-api.com/images/original/MV5BYjc4MzE2NzctNzQyMS00ZmRhLThhOTEtM2NkYjkzZTFjOWZiXkEyXkFqcGdeQXVyNjgzMjQ0MTA@._V1_Ratio0.7273_AL_.jpg    1
https://imdb-api.com/images/original/MV5BMDQzOTVhMDQtMTNkMi00MzE1LTg1ZjktYTYyY2U5OWZjZWMwXkEyXkFqcGdeQXVyNTY0NDkzNDc@._V1_Ratio0.7273_AL_.jpg    1
https://imdb-api.com/images/original/MV5BNjkyZGRjMmItOTY3YS00ZDc5LWFiMGItMGI4Njk5ZTk3NTRlXkEyXkFqcGdeQXVyMTA1OTEwNjE@.

In [22]:
df_titulos["url"].isnull().value_counts()

True     4548
False      10
Name: url, dtype: int64

*La ingesta se debe realizar parcial por limitantes comerciales del API*

In [175]:
# Escribir DataFrame a CSV por 
df_titulos.to_csv('titulos_con_url_7.csv',sep=';')

---

In [27]:
df_titulos_fusion = pd.merge(right=df_titulos_generos, right_on="anime_id",
                             left=df_titulos.loc[:,["anime_id","url"]], left_on="anime_id",
                             how="right")
print(df_titulos_fusion.shape)
df_titulos_fusion.head()

(7492, 54)


,anime_id,url_x,Unnamed: 0,url_y,name,genre,type,episodes,rating,members,...,ShounenAi,SliceofLife,Space,Sports,Supernatural,SuperPower,Thriller,Vampire,Yaoi,Yuri
0,820,None,0,NaN,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110.0,9.11,80679,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30709,None,1,NaN,Kamisama Hajimemashita: Kako-hen,"Comedy, Demons, Fantasy, Shoujo, Supernatural",OVA,4.0,8.64,33422,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,12431,None,2,NaN,Uchuu Kyoudai,"Comedy, Sci-Fi, Seinen, Slice of Life, Space",TV,99.0,8.59,72958,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17389,None,3,NaN,Kingdom 2nd Season,"Action, Historical, Military, Seinen",TV,39.0,8.57,31234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24687,None,4,NaN,Mushishi Zoku Shou: Odoro no Michi,"Adventure, Fantasy, Historical, Mystery, Seine...",Special,1.0,8.54,34011,...,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [28]:
df_titulos_fusion["url"] = np.where( pd.isnull(df_titulos_fusion["url_y"]),
                                     df_titulos_fusion["url_x"],
                                     df_titulos_fusion["url_y"])
df_titulos_fusion["url"].isnull().value_counts()

True     4548
False    2944
Name: url, dtype: int64

In [29]:
df_titulos_fusion.columns

Index(['anime_id', 'url_x', 'Unnamed: 0', 'url_y', 'name', 'genre', 'type',
       'episodes', 'rating', 'members', 'log_members', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'MartialArts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi',
       'SliceofLife', 'Space', 'Sports', 'Supernatural', 'SuperPower',
       'Thriller', 'Vampire', 'Yaoi', 'Yuri', 'url'],
      dtype='object')

In [30]:
df_titulos_fusion.drop(['url_x', 'Unnamed: 0', 'url_y'],axis=1,inplace=True)

In [31]:
# Exportamos a .csv los df de interacciones final
df_titulos_fusion.to_csv('titulos_generos_VF20200519.csv',sep=';')

---
### Principales hallazgos:

Se encuentran posters para 2.944 animes, sin embargo 438 corresponde a imagenes vacías.
Se observa que en todos los generos existen animes con posters encontrado positivamente.

In [35]:
df_titulos_fusion["url"].isnull().value_counts()

True     4548
False    2944
Name: url, dtype: int64

In [36]:
for i in subgenre:
    print (i)
    print(df_titulos_fusion[df_titulos_fusion[i]==1]["url"].isnull().value_counts())

Action
True     1006
False     819
Name: url, dtype: int64
Adventure
True     856
False    706
Name: url, dtype: int64
Cars
False    29
True     15
Name: url, dtype: int64
Comedy
True     1696
False    1070
Name: url, dtype: int64
Dementia
False    79
True     51
Name: url, dtype: int64
Demons
True     137
False     59
Name: url, dtype: int64
Drama
True     599
False    585
Name: url, dtype: int64
Ecchi
True     243
False    153
Name: url, dtype: int64
Fantasy
True     848
False    524
Name: url, dtype: int64
Game
True     68
False    44
Name: url, dtype: int64
Harem
True     99
False    58
Name: url, dtype: int64
Hentai
True     829
False    174
Name: url, dtype: int64
Historical
True     237
False    211
Name: url, dtype: int64
Horror
True     128
False    110
Name: url, dtype: int64
Josei
True     23
False    11
Name: url, dtype: int64
Kids
True     389
False    222
Name: url, dtype: int64
Magic
True     318
False    198
Name: url, dtype: int64
MartialArts
True     105
False     64


In [34]:
df_titulos_fusion["url"].value_counts()

https://imdb-api.com/images/original/nopicture.jpg                                                                                                               438
https://imdb-api.com/images/original/MV5BMGZhOTQ1YmEtODI4NC00MjdkLWFmNjktODIxMzk3OGUwMTk0XkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_Ratio0.7273_AL_.jpg                     10
https://imdb-api.com/images/original/MV5BZmJmYTIwMWUtMWI4ZC00NjA1LTk4OTMtZDgyZGE1ZjVlMWVjL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyMjc4OTQ1OTA@._V1_Ratio0.7273_AL_.jpg      8
https://imdb-api.com/images/original/MV5BNWFkNjQ1ZjAtZDMyZC00NWI3LWE2ZDctZWM2MmUxMTE5MzcwXkEyXkFqcGdeQXVyMTk2MDc1MjQ@._V1_Ratio0.7273_AL_.jpg                      6
https://imdb-api.com/images/original/MV5BNTk1OWE3MmItNDhlYi00NGM4LTkwMWUtM2NkZWJiNjQyNGIxXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_Ratio0.7273_AL_.jpg                      6
https://imdb-api.com/images/original/MV5BMTRmYWMzMTMtNTY2MC00NmEzLThiYWEtYThmZDNhNzgwNGEzXkEyXkFqcGdeQXVyMjc4OTQ1OTA@._V1_Ratio0.7273_AL_.jpg                      6
https://im

---

----